# Exploratory Data Analysis


### Definiera problemet och skapa en helhetsbild
- GOOD

### Få tillgång till datan
- GOOD

### Utforska datan, gör en exploratory data analysis (EDA)
- Todo

### Bearbeta datan
- Todo

### ML-modellering
- Todo

### Presentera din lösning för intressenter
- Todo

### Produktionssättning av modellen och övervakning av implementeringen
- Todo


### Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns



**Load Data and get Head**

In [3]:
data = pd.read_csv('recruitment_data.csv')
data.head()

,Age,Gender,EducationLevel,ExperienceYears,PreviousCompanies,DistanceFromCompany,InterviewScore,SkillScore,PersonalityScore,RecruitmentStrategy,HiringDecision
0,26,1,2,0,3,26.783828,48,78,91,1,1
1,39,1,4,12,3,25.862694,35,68,80,2,1
2,48,0,2,3,2,9.920805,20,67,13,2,0
3,34,1,2,5,2,6.407751,36,27,70,3,0
4,30,0,1,6,1,43.105343,23,52,85,2,0


**Check for null values**

In [4]:
# Print null values (NaN)
data.isnull().sum()

Age                    0
Gender                 0
EducationLevel         0
ExperienceYears        0
PreviousCompanies      0
DistanceFromCompany    0
InterviewScore         0
SkillScore             0
PersonalityScore       0
RecruitmentStrategy    0
HiringDecision         0
dtype: int64